In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.chips['C_chip'].size['size_x']='3mm'
design.chips['C_chip'].size['size_y']='3mm'
design.chips['Q_chip'].size['size_x']='2mm'
design.chips['Q_chip'].size['size_y']='2mm'
design.chips['Q_chip'].size['center_z']='7um'

In [4]:
design.get_chip_z('Q_chip')

'7um'

In [9]:
q0

[name:    Q0
 class:   TransmonPocket6       
 options: 
   'pos_x'             : '-300um',                     
   'pos_y'             : '-500um',                     
   'orientation'       : '0.0',                        
   'chip'              : 'Q_chip',                     
   'layer'             : '1',                          
   'connection_pads'   : {
                         },
   'pad_gap'           : '70um',                       
   'inductor_width'    : '20um',                       
   'pad_width'         : '400um',                      
   'pad_height'        : '100um',                      
   'pocket_width'      : '650um',                      
   'pocket_height'     : '650um',                      
   'hfss_wire_bonds'   : False,                        
   'q3d_wire_bonds'    : False,                        
   'hfss_inductance'   : '10nH',                       
   'hfss_capacitance'  : 0,                            
   'hfss_resistance'   : 0,                     

In [4]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-300um',
                                            pos_y='-500um',
                                            chip='Q_chip',
                                            pad_width='370um',
                                            pad_height='100um',
                                            pad_gap='80um',
                                            pocket_width='680um',
                                            pocket_height='680um'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-120}um",
                                              pos_y=f"{q0y+90}um",
                                              pad_width="132um",
                                              pad_height="100um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [5]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-700um',
                                         pos_y = '800um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '700um',
                                         pos_y = '800um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [6]:
stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='0.65mm',
                                                pos_y='-0.2mm',
                                                orientation='270'))

In [7]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [8]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length
4.29mm

In [9]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [10]:
resonator_length_4=[]
a=find_resonator_length(7.5,10,6,4)
a=a.strip('mm')
a=float(a)-0.25 +0.5-0.23# subtract coupler length, for +0.5Ghz, coupler length
resonator_length_4.append(a)
print(resonator_length_4)


[4.077562247348839]


In [20]:
jogs1=OrderedDict()
jogs1[0]=['R','300um']
jogs1[1]=['R', '300um']
jogs1[2]=['L','700um']
jogs1[3]=['R','200um']
jogs1[4]=['R','700um']
jogs1[5]=['L','200um']
jogs1[6]=['L','770um']


In [21]:
rr0=connect('rr0', 'c0', 'in','stg', 'short',
            resonator_length_4[0],jogs1,jogs, 
            '100um','0um', '944um', '100um')
            

# EPR Analysis

In [22]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [24]:
hfss.start()

INFO 05:13PM [connect_project]: Connecting to Ansys Desktop API...
INFO 05:13PM [load_ansys_project]: 	Opened Ansys App
INFO 05:13PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 05:13PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   KPS
INFO 05:13PM [connect_design]: 	Opened active design
	Design:    Planar [Solution type: Eigenmode]
INFO 05:13PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 05:13PM [connect]: 	Connected to project "KPS" and design "Planar" 😀 



True

In [25]:
hfss.activate_ansys_design("flip7um_V4-", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

05:13PM 20s WARNING [activate_ansys_design]: The design_name=flip7um_V4- was not in active project.  Designs in active project are: 
['Planar', 'Design_q3d', 'flip7um_PlanarDesign', 'flip7um_V2', 'bump', 'flip7um_V3'].  A new design will be added to the project.  
INFO 05:13PM [connect_design]: 	Opened active design
	Design:    flip7um_V4- [Solution type: Eigenmode]
WARNING 05:13PM [connect_setup]: 	No design setup detected.
WARNING 05:13PM [connect_setup]: 	Creating eigenmode default setup.
INFO 05:13PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [27]:
hfss.close()


True

# LOM Analysis

In [30]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [31]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='4.8',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '4.8',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [33]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 04:23PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpfktp5g1o.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 4800000000, Maxwell, 1, False
INFO 04:23PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmph6fdtun2.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4800000000, Maxwell, 1, False
INFO 04:23PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpq9kqk2c9.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4800000000, Maxwell, 2, False
INFO 04:23PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpt4k_7jxa.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4800000000, Maxwell, 3, False
INFO 04:23PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpz9rsu8y8.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4800000000, Maxwell, 4, False
INFO 04:23

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,370.320,-122.820000,-107.020,-75.308000
pad_bot_Q0,-122.820,139.040000,-14.837,-0.097944
pad_top_Q0,-107.020,-14.837000,139.060,-15.702000
quad_coupler_c0,-75.308,-0.097944,-15.702,91.174000


In [34]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.95
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.164396 [GHz]
EC 247.552510 [MHz]
EJ 14.854145 [GHz]
alpha -280.122062 [MHz]
dispersion 1.841278 [KHz]
Lq 10.995564 [nH]
Cq 78.246947 [fF]
T1 94.356370 [us]

**Coupling Properties**

tCqbus1 -7.808330 [fF]
gbus1_in_MHz -103.173825 [MHz]
χ_bus1 -1.658406 [MHz]
1/T1bus1 1686.742967 [Hz]
T1bus1 94.356370 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.552002,288.565483,14.854145,-330.59549,8.805326,[-205.72618478214446],[-11.76385197695549],11.763852,205.726185
2,5.335569,265.22844,14.854145,-301.743362,3.781098,[-114.29860977336497],[-2.6021103986687595],2.602110,114.298610
3,5.227271,253.965821,14.854145,-287.943797,2.41169,[-108.24069198557233],[-1.9944757863675664],1.994476,108.240692
4,5.158315,246.937105,14.854145,-279.372889,1.793205,[-107.08590624356997],[-1.7714665646597878],1.771467,107.085906
5,5.138238,244.911449,14.854145,-276.908632,1.64249,[-103.937421636224],[-1.6229299632867906],1.622930,103.937422
6,5.17655,248.785037,14.854145,-281.623226,1.940885,[-104.44198454463124],[-1.7285590789201177],1.728559,104.441985
7,5.164396,247.55251,14.854145,-280.122062,1.841278,[-103.17382472930872],[-1.6584056689733948],1.658406,103.173825


In [35]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
